IN THIS NOTEBOOK I CREATE A READY TO GO DATASET. EACH ROW REPRESENT A DIFFERENT CHAPTER OF ALL THE PUBLICATION IN THE ORIGINAL DATAFRAME. IN THIS WAY EACH ROW IS FORMED BY THE NAME OF THE PUBLICATION, THE NAME OF THE DATASET(S) USED, THE LABEL(S), THE NAME OF THE CHAPTER AND THE TEXT OF THE CHAPTER. 

In [ ]:
import numpy as np 
import pandas as pd 
import os
import json
import glob
from tqdm.autonotebook import tqdm

In [ ]:
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")

In [ ]:
def read_json(df = 'train'):
    
    if df == 'train':
        path = '../input/coleridgeinitiative-show-us-the-data/train'
        df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
    elif df == 'test':
        path = '../input/coleridgeinitiative-show-us-the-data/test'
        df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
    
    
    files = glob.glob(path + "/*.json")
    
    Ids = []
    headings = []
    content = []
    publication = []
    
    for i in range(0,len(files)):
        Ids.append(files[i][52:-5]) 
    
    pt = []
    dt = []
    dl = []
    cl = []

    for i in df['Id'].unique():
        pt.append((df[df['Id'] == i].pub_title).to_list())
        dt.append((df[df['Id'] == i].dataset_title).to_list())
        dl.append((df[df['Id'] == i].dataset_label).to_list())
        cl.append((df[df['Id'] == i].cleaned_label).to_list())

    df = pd.DataFrame({'Id':df['Id'].unique(),'pub_title':pt,'dataset_title':dt,'dataset_label':dl,'cleaned_label':cl})

    
    for j in range(0,len(files)):
        with open(files[j],'r') as f:
            json_decode = json.load(f)
            for data in range(0,len(json_decode)):
                headings.append(json_decode[data]['section_title'])
                content.append(json_decode[data]['text'])
                publication.append(Ids[j])
    
    contents = pd.DataFrame({'head': headings,'text': content, 'Id' : publication})       
    
    contents = pd.merge(contents,df, how = 'left', on = 'Id')
    contents = contents.drop(columns=["Id"])
    contents = contents[['pub_title', 'dataset_title', 'dataset_label','cleaned_label', 'head', 'text']]
    
    return(contents)

In [ ]:
%%time
read_json('train')